In [ ]:
%pip install llama-index-vector-stores-pinecone
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-openai

In [2]:
%pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.1 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 7.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [17]:
# define pinecone index
import pinecone
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")
pinecone = pinecone.Pinecone(api_key=api_key, environment="us-west1-gcp-free")

# dimensions are for text-embedding-ada-002
# pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")
pinecone_index = pinecone.Index("quickstart")

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Installing plugin inference into Pinecone
Installing plugin infer

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '626eb5368f65fc230fd4c993375bd0e4', 'Date': 'Sat, 31 Aug 2024 08:19:10 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource quickstart not found"},"status":404}


In [32]:
import pinecone
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")
pinecone_var = pinecone.Pinecone(api_key=api_key)

pinecone_index = pinecone_var.create_index(
  name="example-index",
  dimension=1536,
  metric="cosine",
  spec=pinecone.PodSpec(
    environment="gcp-starter",
    pod_type='s1.x1'
  ),
  deletion_protection="disabled"

)


INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Installing plugin inference into Pinecone
Installing plugin infer

In [31]:
# OPTIONAL: delete all
#pinecone_var.delete_index('example-index')

In [33]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [49]:
engine = create_engine('postgresql+psycopg2://user@localhost/llm')
metadata_obj = MetaData()

In [50]:
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [51]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [52]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


In [53]:
%pip install wikipedia

4767.17s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=87fb99f6c21ef7ee1a06c0a37ef179dafec944b085b3f5516bc28c955d1622e3
  Stored in directory: /Users/chuapk/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)